In [1]:
# (9. f(x, y, z) = -1/(5+x^4) + tanh(2y'^2) + sinh^2(z'), (y0, z0) = (1, -1), alpha = pi/4
from typing import Any, List
import numpy as np

ANALYTICAL_RESULT = np.array([0, 1, -1])
X0 = np.array([0, 1, -1])
ALPHA = np.pi / 4
COS_A = np.cos(ALPHA)
SIN_A = np.sin(ALPHA)

def get_y_prime(y: int | float, z: int | float) -> Any:
    return (y - X0[1]) * COS_A + (z - X0[2]) * SIN_A

def get_z_prime(y: int | float, z: int | float) -> Any:
    return -(y - X0[1]) * SIN_A + (z - X0[2]) * COS_A

# Производные функции f по штрихованным координатам для градиента и гессиана
def df_dy_prime(yp: int | float) -> Any:
    return 4 * yp * (1 / np.cosh(2 * yp**2)**2)

def df_dz_prime(zp: int | float) -> Any:
    return 2 * np.sinh(zp) * np.cosh(zp)

def d2f_dy_prime2(yp: int | float) -> Any:
    tanh_term = np.tanh(2 * yp**2)
    sech2_term = 1 / np.cosh(2 * yp**2)**2
    return 4 * sech2_term - 32 * yp**2 * tanh_term * sech2_term

def d2f_dz_prime2(zp: int | float) -> Any:
    return 2 * (np.cosh(zp)**2 + np.sinh(zp)**2)

# Градиент и гессиан
def gradient(X: np.ndarray) -> np.ndarray:
    x, y, z = X
    yp = get_y_prime(y, z)
    zp = get_z_prime(y, z)

    # Производные по штрихованным координатам в текущей точке
    df_dyp = df_dy_prime(yp)
    df_dzp = df_dz_prime(zp)

    # Компоненты градиента
    g_x = (4 * x**3) / (5 + x**4)**2
    g_y = df_dyp * COS_A - df_dzp * SIN_A
    g_z = df_dyp * SIN_A + df_dzp * COS_A

    return np.array([g_x, g_y, g_z])

def hessian(X: np.ndarray) -> np.ndarray:
    x, y, z = X
    yp = get_y_prime(y, z)
    zp = get_z_prime(y, z)

    # Вторые производные по штрихованным координатам в текущей точке
    d2f_dyp2 = d2f_dy_prime2(yp)
    d2f_dzp2 = d2f_dz_prime2(zp)

    # Вторая производная по x
    x4 = x**4
    term1 = (12 * x**2) / (5 + x4)**2
    term2 = (32 * x**6) / (5 + x4)**3
    h_xx = term1 - term2

    # Компоненты гессиана
    h_yy = d2f_dyp2 * COS_A**2 + d2f_dzp2 * SIN_A**2
    h_zz = d2f_dyp2 * SIN_A**2 + d2f_dzp2 * COS_A**2
    h_yz = COS_A * SIN_A * (d2f_dyp2 - d2f_dzp2)

    return np.array([
        [h_xx, 0, 0],
        [0, h_yy, h_yz],
        [0, h_yz, h_zz],
    ])

def newton_method(
    initial_guess: List[int | float], 
    tolerance: float = 1e-10, 
    max_iterations: int = 100,
) -> np.ndarray:
    Xk = np.array(initial_guess, dtype=float)

    print(f"Начальная точка: {Xk}")
    print("-" * 50)
    for i in range(max_iterations):
        grad = gradient(Xk)
        H = hessian(Xk)

        # Проверка на близость к нулю градиента (уже в минимуме)
        grad_norm = np.linalg.norm(grad)
        if grad_norm < tolerance:
            print(f"Градиент достаточно мал. Сходимся на итерации {i+1}.")
            break
        
        # Вычисление шага с регуляризацией
        # Добавляем небольшой шум к диагонали для устойчивости
        regularization = 1e-8
        H_reg = H + np.eye(3) * regularization
        H_inv = np.linalg.inv(H_reg)
        step = H_inv.dot(grad) # умножение матриц

        # Обновление точки
        Xk_next = Xk - step
        
        print(f"Итерация {i+1}:")
        print(f"  Xk     = {Xk}")
        print(f"  grad(f)  = {grad}")
        print(f"  |grad(f)|= {grad_norm:.2e}")
        print(f"  step   = {step}")
        print(f"  Xk+1   = {Xk_next}\n")

        # Проверка условия остановки
        if np.linalg.norm(step) < tolerance:
            print(f"Шаг достаточно мал. Сходимся на итерации {i+1}.")
            Xk = Xk_next
            break
        
        Xk = Xk_next    
        print("-" * 50)
    else: # Если цикл завершился без break
        print("Достигнуто максимальное количество итераций.")
    
    return Xk

if __name__ == "__main__":
    # Выбираю начальную точку, не слишком близкую к решению
    # start_point = [1, 1.1, -0.9]
    start_point = [0, 1, -1]
    # start_point = [0, 0.5, -0.5] # 5 итераций 
    # start_point = [0,0.49, -0.59]
    # start_point = [0, 0.9999999, -0.999999] # 2 итерации
    # start_point = [0, 0.9, -1.1] # 3 итерации
    # start_point = [3, 0, 4]
    
    # Ищем минимум
    min_point = newton_method(start_point)
    
    print("-" * 50)
    print(f"Аналитическое решение: {ANALYTICAL_RESULT}")
    print(f"Найденная точка минимума: {min_point}")

Начальная точка: [ 0.  1. -1.]
--------------------------------------------------
Градиент достаточно мал. Сходимся на итерации 1.
--------------------------------------------------
Аналитическое решение: [ 0  1 -1]
Найденная точка минимума: [ 0.  1. -1.]
